In [1]:
import pandas as pd
import re
from hanspell import spell_checker
from konlpy.tag import Okt

In [19]:
stop_words = [
    "가", "가까스로", "가령", "각", "각각", "각자", "각종", "갖고말하자면", "같다", "같이",
    "개의치않고", "거니와", "거바", "거의", "것", "것과 같이", "것들", "게다가", "게우다",
    "겨우", "견지에서", "결과에 이르다", "결국", "결론을 낼 수 있다", "겸사겸사", "고려하면",
    "고로", "곧", "공동으로", "과", "과연", "관계가 있다", "관계없이", "관련이 있다", "관하여",
    "관한", "관해서는", "구", "구체적으로", "구토하다", "그", "그들", "그때", "그래", "그래도",
    "그래서", "그러나", "그러니", "그러니까", "그러면", "그러므로", "그러한즉", "그런 까닭에",
    "그런데", "그런즉", "그럼", "그럼에도 불구하고", "그렇게 함으로써", "그렇지", "그렇지 않다면",
    "그렇지 않으면", "그렇지만", "그렇지않으면", "그리고", "그리하여", "그만이다", "그에 따르는",
    "그위에", "그저", "그중에서", "그치지 않다", "근거로", "근거하여", "기대여", "기점으로",
    "기준으로", "기타", "까닭으로", "까악", "까지", "까지 미치다", "까지도", "꽈당", "끙끙", "끼익",
    "나", "나머지는", "남들", "남짓", "너", "너희", "너희들", "네", "넷", "년", "논하지 않다",
    "놀라다", "누가 알겠는가", "누구", "다른", "다른 방면으로", "다만", "다섯", "다소", "다수", "다시 말하자면",
    "다시말하면", "다음", "다음에", "다음으로", "단지", "답다", "당신", "당장", "대로 하다", "대하면",
    "대하여", "대해 말하자면", "대해서", "댕그", "더구나", "더군다나", "더라도", "더불어", "더욱더",
    "더욱이는", "도달하다", "도착하다", "동시에", "동안", "된바에야", "된이상", "두번째로", "둘", "둥둥",
    "뒤따라", "뒤이어", "든간에", "들", "등", "등등", "딩동", "따라", "따라서", "따위", "따지지 않다",
    "딱", "때", "때가 되어", "때문에", "또", "또한", "뚝뚝", "라 해도", "령", "로", "로 인하여", "로부터",
    "로써", "륙", "를", "마음대로", "마저", "마저도", "마치", "막론하고", "만 못하다", "만약", "만약에",
    "만은 아니다", "만이 아니다", "만일", "만큼", "말하자면", "말할것도 없고", "매", "매번", "메쓰겁다",
    "몇", "모", "모두", "무렵", "무릎쓰고", "무슨", "무엇", "무엇때문에", "물론", "및", "바꾸어말하면",
    "바꾸어말하자면", "바꾸어서 말하면", "바꾸어서 한다면", "바꿔 말하면", "바로", "바와같이", "밖에 안된다",
    "반대로", "반대로 말하자면", "반드시", "버금", "보는데서", "보다더", "보드득", "본대로", "봐", "봐라",
    "부류의 사람들", "부터", "불구하고", "불문하고", "붕붕", "비걱거리다", "비교적", "비길수 없다", "비로소",
    "비록", "비슷하다", "비추어 보아", "비하면", "뿐만 아니라", "뿐만아니라", "뿐이다", "삐걱", "삐걱거리다",
    "사", "삼", "상대적으로 말하자면", "생각한대로", "설령", "설마", "설사", "셋", "소생", "소인", "솨", "쉿",
    "습니까", "습니다", "시각", "시간", "시작하여", "시초에", "시키다", "실로", "심지어", "아", "아니", "아니나다를가",
    "아니라면", "아니면", "아니었다면", "아래윗", "아무거나", "아무도", "아야", "아울러", "아이", "아이고", "아이구",
    "아이야", "아이쿠", "아하", "아홉", "안 그러면", "않기 위하여", "않기 위해서", "알 수 있다", "알았어", "앗",
    "앞에서", "앞의것", "야", "약간", "양자", "어", "어기여차", "어느", "어느 년도", "어느것", "어느곳", "어느때",
    "어느쪽", "어느해", "어디", "어때", "어떠한", "어떤", "어떤것", "어떤것들", "어떻게", "어떻해", "어이", "어째서",
    "어쨋든", "어쩔수 없다", "어찌", "어찌됏든", "어찌됏어", "어찌하든지", "어찌하여", "언제", "언젠가", "얼마",
    "얼마 안 되는 것", "얼마간", "얼마나", "얼마든지", "얼마만큼", "얼마큼", "엉엉", "에", "에 가서", "에 달려 있다",
    "에 대해", "에 있다", "에 한하다", "에게", "에서", "여", "여기", "여덟", "여러분", "여보시오", "여부", "여섯",
    "여전히", "여차", "연관되다", "연이서", "영", "영차", "옆사람", "예", "예를 들면", "예를 들자면", "예컨대", "예하면",
    "오", "오로지", "오르다", "오자마자", "오직", "오호", "오히려", "와", "와 같은 사람들", "와르르", "와아", "왜",
    "왜냐하면", "외에도", "요만큼", "요만한 것", "요만한걸", "요컨대", "우르르", "우리", "우리들", "우선", "우에 종합한것과같이",
    "운운", "월", "위에서 서술한바와같이", "위하여", "위해서", "윙윙", "육", "으로", "으로 인하여", "으로서", "으로써",
    "을", "응", "응당", "의", "의거하여", "의지하여", "의해", "의해되다", "의해서", "이", "이 되다", "이 때문에",
    "이 밖에", "이 외에", "이 정도의", "이것", "이곳", "이때", "이라면", "이래", "이러이러하다", "이러한", "이런",
    "이럴정도로", "이렇게 많은 것", "이렇게되면", "이렇게말하자면", "이렇구나", "이로 인하여", "이르기까지", "이리하여",
    "이만큼", "이번", "이봐", "이상", "이어서", "이었다", "이와 같다", "이와 같은", "이와 반대로", "이와", "입니다", "됩니다", "이다", "있습니다"
]
# 불용어 리스트 참고 "https://gist.github.com/spikeekips/40eea22ef4a89f629abd87eed535ac6a.js", https://www.ranks.nl/stopwords/korean 

In [11]:
import re

def remove_special_characters_and_emojis(text):
    """
    텍스트에서 특수문자와 이모지를 제거하는 함수
    
    Args:
        text (str): 입력 텍스트
        
    Returns:
        str: 특수문자와 이모지가 제거된 텍스트
    """
    if isinstance(text, str):  # 입력이 문자열인지 확인
        # 정규식으로 특수문자와 이모지 제거
        # [^\w\s]는 특수문자 제거, \u3000-\u303F, \U0001F600-\U0001F64F 등 유니코드 범위는 이모지 제거
        text = re.sub(r'[^\w\s\uAC00-\uD7A3]', '', text)  # 한글, 영어, 숫자, 공백만 남김
        return text
    return text  # 문자열이 아닐 경우 원본 반환

In [54]:
# CSV 파일 읽기
file_path = "경주리뷰멜트.csv"  # 여기에 실제 파일 경로를 넣으세요.
df = pd.read_csv(file_path)

# 특수문자 및 이모지 제거
df["cleaned_review"] = df["review"].apply(remove_special_characters_and_emojis)

# 결과 확인
print(df.head())

          0                                             review  \
0  감은사지삼층석탑  ‘돌이 뭔가 말을 하는거 같다’랬던가, 기가 막힌 감상평이다. 그나저나 저 나무는 ...   
1  감은사지삼층석탑  신문왕의 기운이 느껴진다. 그래, 내가 통일신라의 왕이다. 불국사 탑들에서 느껴지는...   
2  감은사지삼층석탑                                         고양이 있어요 최고   
3  감은사지삼층석탑                조용하고 한적하니 천천히 둘려보기 진짜 좋음특히 일몰이 멋진곳임   
4  감은사지삼층석탑                      신라시대 신문왕때 조성, 문무대왕  수중릉, 이견대.   

                                      cleaned_review  
0  돌이 뭔가 말을 하는거 같다랬던가 기가 막힌 감상평이다 그나저나 저 나무는 언제부터...  
1  신문왕의 기운이 느껴진다 그래 내가 통일신라의 왕이다 불국사 탑들에서 느껴지는 경덕...  
2                                         고양이 있어요 최고  
3                조용하고 한적하니 천천히 둘려보기 진짜 좋음특히 일몰이 멋진곳임  
4                         신라시대 신문왕때 조성 문무대왕  수중릉 이견대  


In [55]:
df

,0,review,cleaned_review
0,감은사지삼층석탑,"‘돌이 뭔가 말을 하는거 같다’랬던가, 기가 막힌 감상평이다. 그나저나 저 나무는 ...",돌이 뭔가 말을 하는거 같다랬던가 기가 막힌 감상평이다 그나저나 저 나무는 언제부터...
1,감은사지삼층석탑,"신문왕의 기운이 느껴진다. 그래, 내가 통일신라의 왕이다. 불국사 탑들에서 느껴지는...",신문왕의 기운이 느껴진다 그래 내가 통일신라의 왕이다 불국사 탑들에서 느껴지는 경덕...
2,감은사지삼층석탑,고양이 있어요 최고,고양이 있어요 최고
3,감은사지삼층석탑,조용하고 한적하니 천천히 둘려보기 진짜 좋음특히 일몰이 멋진곳임,조용하고 한적하니 천천히 둘려보기 진짜 좋음특히 일몰이 멋진곳임
4,감은사지삼층석탑,"신라시대 신문왕때 조성, 문무대왕 수중릉, 이견대.",신라시대 신문왕때 조성 문무대왕 수중릉 이견대
...,...,...,...
2060,황리단길,경주에서 사람구경하고싶으면 여기가믄댐,경주에서 사람구경하고싶으면 여기가믄댐
2061,황리단길,",",
2062,황리단길,",",
2063,황리단길,",",


In [56]:
def remove_stopwords(text, stop_words):
    """
    텍스트 데이터에서 불용어를 제거하는 함수
    
    Args:
        text (str): 입력 텍스트
        stop_words (list): 불용어 리스트
        
    Returns:
        str: 불용어가 제거된 텍스트
    """
    okt = Okt()  # 형태소 분석기
    if isinstance(text, str):  # 입력이 문자열인지 확인
        # 형태소 분석 (토큰화)
        tokens = okt.morphs(text)
        # 불용어 제거
        filtered_tokens = [word for word in tokens if word not in stop_words]
        # 다시 하나의 문장으로 합침
        return ' '.join(filtered_tokens)
    return text  # 문자열이 아닌 경우 원본 반환

In [57]:
# 불용어 제거
df["cleaned_review"] = df["cleaned_review"].apply(lambda x: remove_stopwords(x, stop_words))

# 결과 확인
print(df)

             0                                             review  \
0     감은사지삼층석탑  ‘돌이 뭔가 말을 하는거 같다’랬던가, 기가 막힌 감상평이다. 그나저나 저 나무는 ...   
1     감은사지삼층석탑  신문왕의 기운이 느껴진다. 그래, 내가 통일신라의 왕이다. 불국사 탑들에서 느껴지는...   
2     감은사지삼층석탑                                         고양이 있어요 최고   
3     감은사지삼층석탑                조용하고 한적하니 천천히 둘려보기 진짜 좋음특히 일몰이 멋진곳임   
4     감은사지삼층석탑                      신라시대 신문왕때 조성, 문무대왕  수중릉, 이견대.   
...        ...                                                ...   
2060      황리단길                               경주에서 사람구경하고싶으면 여기가믄댐   
2061      황리단길                                                 ,    
2062      황리단길                                                 ,    
2063      황리단길                                                 ,    
2064      황리단길                       주차가 너무 불편해요 ㅠㅠㅠㅠ 그거 빼면 만족합니당   

                                         cleaned_review  
0        돌이 뭔가 말 하는거 같다랬던 기 막힌 감상 평이 다 그나저나 저 나무 는 있던거야  
1     신문왕 기운 느껴진다 내 통일신라 왕 불국사 탑 느껴지는 경덕왕 시절 예술 적 극치...

In [58]:
df = df.drop(['review'], axis=1)

In [59]:
df

,0,cleaned_review
0,감은사지삼층석탑,돌이 뭔가 말 하는거 같다랬던 기 막힌 감상 평이 다 그나저나 저 나무 는 있던거야
1,감은사지삼층석탑,신문왕 기운 느껴진다 내 통일신라 왕 불국사 탑 느껴지는 경덕왕 시절 예술 적 극치...
2,감은사지삼층석탑,고양이 있어요 최고
3,감은사지삼층석탑,조용하고 한적하니 천천히 둘려 보기 진짜 좋음 특히 일몰 멋진 곳임
4,감은사지삼층석탑,신라시대 신문왕 조성 문무대왕 수중 릉 이견 대
...,...,...
2060,황리단길,경주 사람 구경 하고 싶으면 믄 댐
2061,황리단길,
2062,황리단길,
2063,황리단길,


In [60]:
import re

def keep_korean_only(text):
    """
    텍스트에서 한글만 남기는 함수
    
    Args:
        text (str): 입력 텍스트
        
    Returns:
        str: 한글만 남은 텍스트
    """
    if isinstance(text, str):  # 입력이 문자열인지 확인
        return re.sub(r'[^가-힣\s]', '', text)  # 한글과 공백만 남김
    return text  # 문자열이 아닐 경우 원본 반환


In [61]:
# 한글만 남기기
df["cleaned_review"] = df["cleaned_review"].apply(keep_korean_only)

# 결과 확인
print(df)

             0                                     cleaned_review
0     감은사지삼층석탑     돌이 뭔가 말 하는거 같다랬던 기 막힌 감상 평이 다 그나저나 저 나무 는 있던거야
1     감은사지삼층석탑  신문왕 기운 느껴진다 내 통일신라 왕 불국사 탑 느껴지는 경덕왕 시절 예술 적 극치...
2     감은사지삼층석탑                                         고양이 있어요 최고
3     감은사지삼층석탑              조용하고 한적하니 천천히 둘려 보기 진짜 좋음 특히 일몰 멋진 곳임
4     감은사지삼층석탑                         신라시대 신문왕 조성 문무대왕 수중 릉 이견 대
...        ...                                                ...
2060      황리단길                                경주 사람 구경 하고 싶으면 믄 댐
2061      황리단길                                                   
2062      황리단길                                                   
2063      황리단길                                                   
2064      황리단길                            주차 너무 불편해요  그거 빼면 만족합니당

[2065 rows x 2 columns]


In [62]:
# 공백인 행 삭제
df = df[df["cleaned_review"].str.strip() != ""].reset_index(drop=True)


In [63]:
df.to_csv('경주_2_토큰화.csv',encoding='UTF-8-sig',index=False)